# **Preply Web Scrape and Upload to Firebase**

### Task Description
<p>I need to write a python script which will scrape records from a website (each record has about ten text fields and two images which need to be downloaded to a local folder) and save all the text information to a pandas data frame along with the image file names. Each record on the website has a unique id in the url string which can be used as the index/primary key. I think using Selenium with ChromeDriver will be the easiest way to do this because I will have to login through the browser window and navigate to the main page and pick some options before starting the scraping loop.

The second part of the project will be to push the scraped information and images to my firebase project. Since each record has a unique identifier it should be pretty straight forward to use that as the primary key to save the text to firestore. Also the images will need to be uploaded to firebase storage and links to the images saved as a field in firestore for each record.</p>

In [21]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import re

In [84]:
CURRENCY_BUTTON = "#__next > section > div._15uGWh._3rnoor.X4eHUe._1o4EVj.hUFwKI._1-5zEl._2x1aeM._1WH1_Q._2JlASK.GWHMDM._1kf68Q._3nnn2i > div.styles_HeaderDropdownWrapper__uIOg5 > button"
URL = "https://preply.com/en/online/english-tutors?page={0}"
n_of_pages_selector = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > div > div > ul > li:nth-child(5) > a"
def safe_execute(error_message, exception, function, *args):
    try:
        return function(*args)
    except exception as e:
        print(e)
        return error_message
to_int = lambda raw_string: int(''.join(re.findall(r'\d+', raw_string)))

class Selectors:
    #__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > div:nth-child(15) > aside > div > div:nth-child(3)
    #__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > div:nth-child(15) > aside > div > div:nth-child(3)
    tutor = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0})"
    tutor_basic_info = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_SearchCardHeading__fL9u3"
    tutor_country = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_SearchCardHeading__fL9u3 > span > img"
    is_newly_joined = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(1) > div > span"
    tutor_rating = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(1) > div"
    thumbnail_img = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div.styles_SearchCardAvatarWrapper__u_zNw > div > a > picture > img"
    price = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(2) > div > div.styles_PriceIndicatorPrice__vz9WR > div.styles_PriceIndicatorValue__ndpfb.styles_PriceIndicatorValueSecond__Py5XD"
    lesson_duration = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(2) > div.styles_CoreIndicatorsBlockWrapper__JrZw6 > div > div > div:nth-child(2) > div > div.styles_PriceIndicatorDuration__GKnmh"
    lessons_and_students = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div._15uGWh._3rnoor.X4eHUe._1o4EVj.qh7on_._22lwVW._2x1aeM._1WH1_Q._2JlASK.GWHMDM"
    tutor_languages = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_SpeaksWrapper__ssRz1"
    tutor_languages_more = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_SpeaksWrapper__ssRz1 > ul > span.styles_MoreSubjects__eJRX9"
    tutor_description = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_TutorDescriptionBlock__Th9E_ > div > div"
    tutor_description_more = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0}) > section > div > div:nth-child(3) > div.styles_InfoWrap__CBt_k > div.styles_TutorDescriptionBlock__Th9E_ > div > span > span"

In [3]:
# Initialize Chrome Driver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")
chrome_driver_path="C:\Program Files\Google\Chrome\Application\chromedriver.exe"
driver = webdriver.Chrome(options=options)
driver.get(URL.format(1))
sleep(2)
print("Driver successfully initialized.")

Driver successfully initialized.


In [4]:
### Selecting Currency 
from selenium.webdriver.support.ui import Select

def refresh_currency():
    driver.find_element(By.CSS_SELECTOR,CURRENCY_BUTTON).click()
    select = Select(driver.find_element(By.NAME, 'currency'))
    sleep(1)
    select.select_by_visible_text("EUR")
    sleep(1)
    driver.find_element(By.CSS_SELECTOR,CURRENCY_BUTTON).click()
    select = Select(driver.find_element(By.NAME, 'currency'))
    select.select_by_visible_text("USD")

try:
    refresh_currency()
    print("Currency successfully refreshed.")
except Exception as e:
    print(f"Could not refresh currency, try again. {e}")

Currency successfully refreshed.


In [5]:
n_of_pages = driver.find_element(By.CSS_SELECTOR, n_of_pages_selector).text

In [96]:
previous_tutor_name = None
for tutor_index in range(1,6):
    ## Validating Collected Selectors
    try:
        if previous_tutor_name != tutor_name:
            pass
        elif  previous_tutor_name == tutor_name:
            continue
        tutor_basic_info = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_basic_info.format(tutor_index))
        tutor_name = tutor_basic_info.text.split('\n')[0]
        previous_tutor_name = tutor_name
        tutor_teaches = tutor_basic_info.text.split('\n')[1]
    except NoSuchElementException:
        pass
    try:
        driver.find_element(By.CSS_SELECTOR,Selectors.tutor_description_more.format(tutor_index)).click()
        sleep(0.5)
    except:
        print("Description is already fully visible - Nothing to click.")

    try:
        driver.find_element(By.CSS_SELECTOR,Selectors.tutor_languages_more.format(tutor_index)).click()
        sleep(0.5)
    except:
        print("No additional Languages - Nothing to click.")


    try:
        price = driver.find_element(By.CSS_SELECTOR,Selectors.price.format(tutor_index)).text
    except NoSuchElementException:
        pass

    try:
        lesson_duration = driver.find_element(By.CSS_SELECTOR,Selectors.lesson_duration.format(tutor_index)).text
    except NoSuchElementException:
        pass

    try:
        is_newly_joined = driver.find_element(By.CSS_SELECTOR,Selectors.is_newly_joined.format(14)).text
        is_newly_joined = True
    except NoSuchElementException:
        is_newly_joined = False

    try:
        tutor_rating = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_rating.format(tutor_index)).text
        rating = tutor_rating.split('\n')[0]
        try:
            n_of_reviews = to_int(tutor_rating.split('\n')[1])
        except:
            message = "Not rated yet"
    except NoSuchElementException:
        message = "Not rated yet"

    try:
        thumbnail_img = driver.find_element(By.CSS_SELECTOR,Selectors.thumbnail_img.format(tutor_index)).get_attribute('src')
    except NoSuchElementException:
        pass

    try:
        tutor_country =  driver.find_element(By.CSS_SELECTOR,Selectors.tutor_country.format(tutor_index)).get_attribute('alt')
    except NoSuchElementException:
        pass

    try:
        lessons_and_students = driver.find_element(By.CSS_SELECTOR,Selectors.lessons_and_students.format(tutor_index)).text
        lessons = to_int(lessons_and_students.split("•")[0])
        try:
            students = to_int(lessons_and_students.split("•")[1])
        except:
            print("No Students yet.")
    except NoSuchElementException:
        pass

    try:
        languages = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_languages.format(tutor_index)).text
    except NoSuchElementException:
        pass

    try:
        description = driver.find_element(By.CSS_SELECTOR,Selectors.tutor_description.format(tutor_index)).text
    except NoSuchElementException:
        pass
    print(
    { 
    "tutor_name":tutor_name
    ,"tutor_teaches":tutor_teaches
    ,"thumbnail_img":thumbnail_img
    ,"tutor_country":tutor_country
    ,"price":price
    ,"lesson_duration":lesson_duration
    ,"is_newly_joined":is_newly_joined
    ,"rating":rating
    ,"n_of_reviews":n_of_reviews
    ,"lessons":lessons
    ,"students":students
    ,"languages":languages
    ,"description":description}
    )

Description is already fully visible - Nothing to click.
No additional Languages - Nothing to click.
No Students yet.
{'tutor_name': 'Brianna C.', 'tutor_teaches': 'English', 'thumbnail_img': 'https://avatars.preply.com/i/logos/i/logos/avatar_mm6njhw04lb.jpg?d=160x160&f=jpeg', 'tutor_country': 'United States of America', 'price': '30', 'lesson_duration': '50-min lesson', 'is_newly_joined': False, 'rating': 'Newly joined', 'n_of_reviews': 18, 'lessons': 2, 'students': 1082, 'languages': 'Speaks:\nEnglishNative Chinese (Mandarin)Beginner SpanishUpper-Intermediate', 'description': "Licensed English teacher with 7 years of experience with both kids and adults Hello, my name is Teacher Bri and I am from the USA. I enjoy traveling and learning new languages, so I can't wait to help you learn English!\nSome other things I enjoy are reading, cooking, yoga, and learning about new cultures. Do we have any of the same interests?"}


In [61]:
full_list.text.split("Speaks:")

['Times are shown in your local timezone\nMon\nTue\nWed\nThu\nFri\nSat\nSun\nMorning\n06:00—12:00\nAfternoon\n12:00—18:00\nEvening\n18:00—24:00\nNight\n00:00—06:00\nView full schedule\nBrianna C.\nEnglish\nNewly joined\n$\n30\n50-min lesson\n2 active students\n',
 "\nEnglishNative Chinese (Mandarin)Beginner+1\nLicensed English teacher with 7 years of experience with both kids and adults Hello, my name is Teacher Bri and I am from the USA. I enjoy traveling and learning new languages, so I can't wait to help you learn English!\nSome other things I enjoy are reading, cooking, yoga, and learning about new cultures. Do we have any of the same interests?\n\nRead more\nBook trial lesson\nMessage\nSiriporn K.\nEnglish\nNewly joined\n$\n13\n50-min lesson\n2 active students\n",
 '\nEnglishProficient ThaiNative\nAn English teacher with 15 years of experience I am half Hainanese and half Thai. I was born and raised in Bangkok, Thailand. I enjoy watching English football. I support Manchester Unit